In [3]:
"""
Manual Implementation of Edge Detection Algorithms
- Canny Edge Detector
- Marr-Hildreth (LoG) Edge Detector
With evaluation against ground truth edge maps
"""

import numpy as np
import cv2
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from scipy.io import loadmat

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def normalize_image(image):
    """Normalize image to [0, 1] range"""
    return image.astype(np.float32) / 255.0

def read_image_grayscale(image_path):
    """Read image and convert to grayscale"""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")

    # Convert to grayscale if needed
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return normalize_image(image)

def read_ground_truth(gt_path):
    """Read ground truth from .mat file or image file"""
    if str(gt_path).endswith('.mat'):
        # Load MATLAB file
        try:
            mat_data = loadmat(str(gt_path))
            # Common keys in ground truth .mat files
            possible_keys = ['groundTruth', 'gt', 'edge', 'edges', 'Boundaries', 'boundaries']

            # Try to find the ground truth data
            gt_data = None
            for key in mat_data.keys():
                if not key.startswith('__'):  # Skip metadata keys
                    gt_data = mat_data[key]
                    break

            if gt_data is None:
                raise ValueError("Could not find ground truth data in .mat file")

            # Handle different data structures
            if isinstance(gt_data, np.ndarray):
                if gt_data.dtype == np.object_:
                    # Nested structure, extract first element
                    gt_data = gt_data[0, 0]
                    # Try to find 'Boundaries' or 'Segmentation' field
                    if hasattr(gt_data, 'dtype') and gt_data.dtype.names:
                        for field in gt_data.dtype.names:
                            if 'bound' in field.lower() or 'seg' in field.lower():
                                gt_data = gt_data[field][0, 0]
                                break

                # Ensure it's a 2D array
                while len(gt_data.shape) > 2:
                    gt_data = gt_data[0]

                gt_image = gt_data.astype(np.float32)

                # Normalize to [0, 1]
                if gt_image.max() > 1:
                    gt_image = gt_image / gt_image.max()

                return gt_image
            else:
                raise ValueError("Unexpected ground truth data format")

        except Exception as e:
            raise ValueError(f"Error reading .mat file {gt_path}: {str(e)}")
    else:
        # Regular image file
        return read_image_grayscale(str(gt_path))

# ============================================================================
# GAUSSIAN SMOOTHING
# ============================================================================

def create_gaussian_kernel(size=5, sigma=1.0):
    """Create a Gaussian kernel manually"""
    kernel = np.zeros((size, size))
    center = size // 2

    # Generate Gaussian kernel
    sum_val = 0
    for i in range(size):
        for j in range(size):
            x, y = i - center, j - center
            kernel[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma**2))
            sum_val += kernel[i, j]

    # Normalize
    kernel /= sum_val
    return kernel

def convolve2d(image, kernel):
    """Manual 2D convolution implementation"""
    img_height, img_width = image.shape
    kernel_height, kernel_width = kernel.shape

    pad_h = kernel_height // 2
    pad_w = kernel_width // 2

    # Pad image
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')

    # Output image
    output = np.zeros_like(image)

    # Perform convolution
    for i in range(img_height):
        for j in range(img_width):
            region = padded[i:i+kernel_height, j:j+kernel_width]
            output[i, j] = np.sum(region * kernel)

    return output

def gaussian_blur(image, kernel_size=5, sigma=1.0):
    """Apply Gaussian blur to image"""
    kernel = create_gaussian_kernel(kernel_size, sigma)
    return convolve2d(image, kernel)

# ============================================================================
# GRADIENT COMPUTATION
# ============================================================================

def compute_gradients(image):
    """Compute gradient magnitude and direction using Sobel operators"""
    # Sobel kernels
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=np.float32)

    sobel_y = np.array([[-1, -2, -1],
                        [ 0,  0,  0],
                        [ 1,  2,  1]], dtype=np.float32)

    # Compute gradients
    grad_x = convolve2d(image, sobel_x)
    grad_y = convolve2d(image, sobel_y)

    # Compute magnitude and direction
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    direction = np.arctan2(grad_y, grad_x)

    return magnitude, direction

# ============================================================================
# NON-MAXIMUM SUPPRESSION
# ============================================================================

def non_maximum_suppression(magnitude, direction):
    """Apply non-maximum suppression to gradient magnitude"""
    height, width = magnitude.shape
    suppressed = np.zeros_like(magnitude)

    # Convert angle to degrees and normalize to [0, 180]
    angle = np.rad2deg(direction) % 180

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            # Get angle sector (0, 45, 90, 135)
            q = 255
            r = 255

            # 0 degrees
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            # 45 degrees
            elif 22.5 <= angle[i, j] < 67.5:
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            # 90 degrees
            elif 67.5 <= angle[i, j] < 112.5:
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            # 135 degrees
            elif 112.5 <= angle[i, j] < 157.5:
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]

            # Suppress non-maximum
            if magnitude[i, j] >= q and magnitude[i, j] >= r:
                suppressed[i, j] = magnitude[i, j]

    return suppressed

# ============================================================================
# DOUBLE THRESHOLDING AND EDGE TRACKING
# ============================================================================

def double_threshold(image, low_ratio=0.05, high_ratio=0.15):
    """Apply double thresholding"""
    high_threshold = image.max() * high_ratio
    low_threshold = high_threshold * low_ratio

    strong = 1.0
    weak = 0.5

    result = np.zeros_like(image)

    strong_i, strong_j = np.where(image >= high_threshold)
    weak_i, weak_j = np.where((image <= high_threshold) & (image >= low_threshold))

    result[strong_i, strong_j] = strong
    result[weak_i, weak_j] = weak

    return result, weak, strong

def edge_tracking_by_hysteresis(image, weak=0.5, strong=1.0):
    """Track edges by hysteresis"""
    height, width = image.shape

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            if image[i, j] == weak:
                # Check 8-connected neighbors
                if ((image[i+1, j-1] == strong) or (image[i+1, j] == strong) or
                    (image[i+1, j+1] == strong) or (image[i, j-1] == strong) or
                    (image[i, j+1] == strong) or (image[i-1, j-1] == strong) or
                    (image[i-1, j] == strong) or (image[i-1, j+1] == strong)):
                    image[i, j] = strong
                else:
                    image[i, j] = 0

    return image

# ============================================================================
# CANNY EDGE DETECTOR
# ============================================================================

def canny_edge_detector(image, sigma=1.0, low_threshold=0.05, high_threshold=0.15):
    """
    Manual implementation of Canny edge detector

    Steps:
    1. Gaussian smoothing
    2. Gradient computation
    3. Non-maximum suppression
    4. Double thresholding
    5. Edge tracking by hysteresis
    """
    # Step 1: Gaussian smoothing
    smoothed = gaussian_blur(image, kernel_size=5, sigma=sigma)

    # Step 2: Compute gradients
    magnitude, direction = compute_gradients(smoothed)

    # Step 3: Non-maximum suppression
    suppressed = non_maximum_suppression(magnitude, direction)

    # Step 4: Double thresholding
    thresholded, weak, strong = double_threshold(suppressed, low_threshold, high_threshold)

    # Step 5: Edge tracking by hysteresis
    edges = edge_tracking_by_hysteresis(thresholded, weak, strong)

    # Convert to binary (0 or 1)
    edges = (edges == strong).astype(np.float32)

    return edges

# ============================================================================
# LAPLACIAN OF GAUSSIAN (LoG)
# ============================================================================

def create_log_kernel(size=9, sigma=1.0):
    """Create Laplacian of Gaussian (LoG) kernel"""
    kernel = np.zeros((size, size))
    center = size // 2

    # Generate LoG kernel
    for i in range(size):
        for j in range(size):
            x = i - center
            y = j - center

            # LoG formula: -1/(πσ^4) * [1 - (x^2+y^2)/(2σ^2)] * exp(-(x^2+y^2)/(2σ^2))
            r_squared = x**2 + y**2
            sigma_squared = sigma**2

            gaussian = np.exp(-r_squared / (2 * sigma_squared))
            kernel[i, j] = -(1 / (np.pi * sigma_squared**2)) * \
                          (1 - r_squared / (2 * sigma_squared)) * gaussian

    # Normalize to sum to zero
    kernel = kernel - np.mean(kernel)

    return kernel

def zero_crossing(image):
    """Detect zero crossings in LoG filtered image"""
    height, width = image.shape
    zero_crossings = np.zeros_like(image)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            # Check for zero crossing by looking at sign changes
            neighbors = [
                image[i-1, j], image[i+1, j],
                image[i, j-1], image[i, j+1],
                image[i-1, j-1], image[i+1, j+1],
                image[i-1, j+1], image[i+1, j-1]
            ]

            # If center pixel is positive and has negative neighbor (or vice versa)
            if image[i, j] > 0:
                if any(n < 0 for n in neighbors):
                    zero_crossings[i, j] = 1
            elif image[i, j] < 0:
                if any(n > 0 for n in neighbors):
                    zero_crossings[i, j] = 1

    return zero_crossings

def marr_hildreth_edge_detector(image, sigma=1.0, threshold=0.01):
    """
    Manual implementation of Marr-Hildreth (LoG) edge detector

    Steps:
    1. Apply LoG filter
    2. Find zero crossings
    3. Threshold based on gradient magnitude
    """
    # Step 1: Apply LoG filter
    kernel_size = int(6 * sigma + 1)
    if kernel_size % 2 == 0:
        kernel_size += 1

    log_kernel = create_log_kernel(kernel_size, sigma)
    log_filtered = convolve2d(image, log_kernel)

    # Step 2: Find zero crossings
    edges = zero_crossing(log_filtered)

    # Step 3: Threshold based on gradient magnitude
    magnitude, _ = compute_gradients(image)
    threshold_value = np.max(magnitude) * threshold
    edges = edges * (magnitude > threshold_value)

    return edges.astype(np.float32)

# ============================================================================
# EVALUATION METRICS
# ============================================================================

def calculate_metrics(detected_edges, ground_truth):
    """Calculate Precision, Recall, and F1 Score"""
    # Flatten arrays
    detected = detected_edges.flatten() > 0.5
    gt = ground_truth.flatten() > 0.5

    # Calculate True Positives, False Positives, False Negatives
    TP = np.sum(detected & gt)
    FP = np.sum(detected & ~gt)
    FN = np.sum(~detected & gt)

    # Calculate metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'TP': TP,
        'FP': FP,
        'FN': FN
    }

# ============================================================================
# VISUALIZATION
# ============================================================================

def visualize_results(original, canny_edges, log_edges, ground_truth,
                     canny_metrics, log_metrics, save_path=None):
    """Visualize edge detection results"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))

    # Original image
    axes[0, 0].imshow(original, cmap='gray')
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')

    # Canny edges
    axes[0, 1].imshow(canny_edges, cmap='gray')
    axes[0, 1].set_title(f'Canny Edge Detection\n' +
                         f'P={canny_metrics["precision"]:.3f}, ' +
                         f'R={canny_metrics["recall"]:.3f}, ' +
                         f'F1={canny_metrics["f1_score"]:.3f}')
    axes[0, 1].axis('off')

    # Marr-Hildreth edges
    axes[1, 0].imshow(log_edges, cmap='gray')
    axes[1, 0].set_title(f'Marr-Hildreth (LoG) Edge Detection\n' +
                         f'P={log_metrics["precision"]:.3f}, ' +
                         f'R={log_metrics["recall"]:.3f}, ' +
                         f'F1={log_metrics["f1_score"]:.3f}')
    axes[1, 0].axis('off')

    # Ground truth
    axes[1, 1].imshow(ground_truth, cmap='gray')
    axes[1, 1].set_title('Ground Truth')
    axes[1, 1].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

# ============================================================================
# MAIN PROCESSING FUNCTION
# ============================================================================

def process_dataset(images_folder, ground_truth_folder, output_folder):
    """Process all images in the dataset"""

    # Create output directories
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    canny_output = output_folder / 'canny_edges'
    log_output = output_folder / 'marr_hildreth_edges'
    comparison_output = output_folder / 'comparisons'

    canny_output.mkdir(exist_ok=True)
    log_output.mkdir(exist_ok=True)
    comparison_output.mkdir(exist_ok=True)

    # Get list of images
    images_folder = Path(images_folder)
    ground_truth_folder = Path(ground_truth_folder)

    image_files = sorted(list(images_folder.glob('*.jpg')) +
                        list(images_folder.glob('*.png')) +
                        list(images_folder.glob('*.bmp')))

    if len(image_files) == 0:
        print(f"No images found in {images_folder}")
        return None, None

    print(f"Found {len(image_files)} images to process")

    # Get list of ground truth files for matching
    gt_files = {}
    for gt_path in ground_truth_folder.glob('*'):
        if gt_path.is_file():
            # Store with stem (filename without extension) as key
            gt_files[gt_path.stem] = gt_path

    print(f"Found {len(gt_files)} ground truth files")

    # Store results
    results = []
    skipped = 0

    # Process each image
    for img_path in tqdm(image_files, desc="Processing images"):
        try:
            # Read image
            image = read_image_grayscale(str(img_path))

            # Try to find matching ground truth
            # Remove 'img' suffix if present and try different variations
            img_stem = img_path.stem

            # Try exact match first
            gt_path = None
            if img_stem in gt_files:
                gt_path = gt_files[img_stem]
            # Try without 'img' suffix
            elif img_stem.endswith('img'):
                stem_without_img = img_stem[:-3]
                if stem_without_img in gt_files:
                    gt_path = gt_files[stem_without_img]
            # Try adding 'img' suffix
            else:
                stem_with_img = img_stem + 'img'
                if stem_with_img in gt_files:
                    gt_path = gt_files[stem_with_img]

            if gt_path is None:
                skipped += 1
                if skipped <= 10:  # Only print first 10 to avoid clutter
                    print(f"No matching ground truth for {img_path.name}")
                continue

            ground_truth = read_ground_truth(gt_path)

            # Apply Canny edge detection
            canny_edges = canny_edge_detector(image, sigma=1.4,
                                             low_threshold=0.05,
                                             high_threshold=0.15)

            # Apply Marr-Hildreth edge detection
            log_edges = marr_hildreth_edge_detector(image, sigma=2.0,
                                                    threshold=0.01)

            # Calculate metrics
            canny_metrics = calculate_metrics(canny_edges, ground_truth)
            log_metrics = calculate_metrics(log_edges, ground_truth)

            # Save edge maps
            cv2.imwrite(str(canny_output / img_path.name),
                       (canny_edges * 255).astype(np.uint8))
            cv2.imwrite(str(log_output / img_path.name),
                       (log_edges * 255).astype(np.uint8))

            # Visualize and save comparison
            visualize_results(image, canny_edges, log_edges, ground_truth,
                            canny_metrics, log_metrics,
                            save_path=comparison_output / f"comparison_{img_path.stem}.png")

            # Store results
            results.append({
                'image': img_path.name,
                'ground_truth': gt_path.name,
                'canny_precision': canny_metrics['precision'],
                'canny_recall': canny_metrics['recall'],
                'canny_f1': canny_metrics['f1_score'],
                'log_precision': log_metrics['precision'],
                'log_recall': log_metrics['recall'],
                'log_f1': log_metrics['f1_score']
            })

        except Exception as e:
            print(f"Error processing {img_path.name}: {str(e)}")
            continue

    if skipped > 10:
        print(f"Skipped {skipped} images total (no matching ground truth)")

    # Check if we have any results
    if len(results) == 0:
        print("\n" + "="*60)
        print("ERROR: No images were successfully processed!")
        print("="*60)
        print("\nPossible issues:")
        print("1. Ground truth filenames don't match image filenames")
        print("2. Check the naming pattern in both folders")
        print("\nExample image files found:")
        for img in list(image_files)[:5]:
            print(f"  - {img.name}")
        print("\nExample ground truth files found:")
        for gt_name in list(gt_files.keys())[:5]:
            print(f"  - {gt_name}")
        return None, None

    # Create results dataframe
    df = pd.DataFrame(results)

    # Calculate average metrics
    avg_metrics = {
        'Canny Precision': df['canny_precision'].mean(),
        'Canny Recall': df['canny_recall'].mean(),
        'Canny F1': df['canny_f1'].mean(),
        'LoG Precision': df['log_precision'].mean(),
        'LoG Recall': df['log_recall'].mean(),
        'LoG F1': df['log_f1'].mean()
    }

    # Save results
    df.to_csv(output_folder / 'detailed_results.csv', index=False)

    # Print summary
    print("\n" + "="*60)
    print("EDGE DETECTION RESULTS SUMMARY")
    print("="*60)
    print(f"\nProcessed {len(df)} images successfully")
    print(f"Skipped {skipped} images (no matching ground truth)\n")
    print("Average Metrics:")
    print("-" * 60)
    for metric, value in avg_metrics.items():
        print(f"{metric:20s}: {value:.4f}")
    print("="*60)

    # Create summary visualization
    create_summary_plot(df, output_folder)

    return df, avg_metrics

def create_summary_plot(df, output_folder):
    """Create summary comparison plot"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    metrics = ['precision', 'recall', 'f1']
    titles = ['Precision', 'Recall', 'F1 Score']

    for idx, (metric, title) in enumerate(zip(metrics, titles)):
        canny_values = df[f'canny_{metric}'].values
        log_values = df[f'log_{metric}'].values

        x = np.arange(len(canny_values))
        width = 0.35

        axes[idx].bar(x - width/2, canny_values, width, label='Canny', alpha=0.8)
        axes[idx].bar(x + width/2, log_values, width, label='Marr-Hildreth', alpha=0.8)

        axes[idx].set_xlabel('Image Index')
        axes[idx].set_ylabel(title)
        axes[idx].set_title(f'{title} Comparison')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(output_folder / 'metrics_comparison.png', dpi=150, bbox_inches='tight')
    plt.close()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function"""

    # Mount Google Drive if not already mounted
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
    except:
        pass

    # Define paths - UPDATE THESE TO MATCH YOUR ACTUAL FOLDER STRUCTURE
    # Try different possible paths
    possible_base_paths = [
        "/content/drive/MyDrive/DIP_A-02_Images/Images",
        "/content/drive/MyDrive/DIP_A-02_Images",
        "/content/DIP_A-02_Images/Images",
        "/content/DIP_A-02_Images"
    ]

    base_path = None
    for path in possible_base_paths:
        if os.path.exists(path):
            base_path = path
            break

    if base_path is None:
        print("Error: Could not find the dataset folder. Please check the path.")
        print("Tried the following paths:")
        for path in possible_base_paths:
            print(f"  - {path}")
        print("\nPlease update the paths in the main() function to match your folder structure.")
        return

    images_folder = os.path.join(base_path, "images")
    ground_truth_folder = os.path.join(base_path, "ground_truth")
    output_folder = os.path.join(os.path.dirname(base_path), "output")

    print("="*60)
    print("MANUAL EDGE DETECTION IMPLEMENTATION")
    print("="*60)
    print(f"\nBase path: {base_path}")
    print(f"Images folder: {images_folder}")
    print(f"Ground truth folder: {ground_truth_folder}")
    print(f"Output folder: {output_folder}\n")

    # Check if folders exist
    if not os.path.exists(images_folder):
        print(f"Error: Images folder not found at {images_folder}")
        print(f"\nChecking what exists in {base_path}:")
        if os.path.exists(base_path):
            contents = os.listdir(base_path)
            print(f"Contents: {contents}")
        return

    if not os.path.exists(ground_truth_folder):
        print(f"Error: Ground truth folder not found at {ground_truth_folder}")
        print(f"\nChecking what exists in {base_path}:")
        if os.path.exists(base_path):
            contents = os.listdir(base_path)
            print(f"Contents: {contents}")
        return

    # Process dataset
    result = process_dataset(images_folder, ground_truth_folder, output_folder)

    if result is None or result[0] is None:
        print("\nProcessing failed. Please check the error messages above.")
        return

    df, avg_metrics = result

    print(f"\nResults saved to: {output_folder}")
    print("Output folders created:")
    print(f"  - Canny edges: {output_folder}/canny_edges")
    print(f"  - Marr-Hildreth edges: {output_folder}/marr_hildreth_edges")
    print(f"  - Comparisons: {output_folder}/comparisons")
    print(f"  - Detailed results CSV: {output_folder}/detailed_results.csv")
    print(f"  - Summary plot: {output_folder}/metrics_comparison.png")

if __name__ == "__main__":
    main()

Mounted at /content/drive
MANUAL EDGE DETECTION IMPLEMENTATION

Base path: /content/drive/MyDrive/DIP_A-02_Images/Images
Images folder: /content/drive/MyDrive/DIP_A-02_Images/Images/images
Ground truth folder: /content/drive/MyDrive/DIP_A-02_Images/Images/ground_truth
Output folder: /content/drive/MyDrive/DIP_A-02_Images/output

Found 200 images to process
Found 200 ground truth files


Processing images: 100%|██████████| 200/200 [26:28<00:00,  7.94s/it]



EDGE DETECTION RESULTS SUMMARY

Processed 200 images successfully
Skipped 0 images (no matching ground truth)

Average Metrics:
------------------------------------------------------------
Canny Precision     : 0.3163
Canny Recall        : 0.1470
Canny F1            : 0.1766
LoG Precision       : 0.2982
LoG Recall          : 0.6095
LoG F1              : 0.3665

Results saved to: /content/drive/MyDrive/DIP_A-02_Images/output
Output folders created:
  - Canny edges: /content/drive/MyDrive/DIP_A-02_Images/output/canny_edges
  - Marr-Hildreth edges: /content/drive/MyDrive/DIP_A-02_Images/output/marr_hildreth_edges
  - Comparisons: /content/drive/MyDrive/DIP_A-02_Images/output/comparisons
  - Detailed results CSV: /content/drive/MyDrive/DIP_A-02_Images/output/detailed_results.csv
  - Summary plot: /content/drive/MyDrive/DIP_A-02_Images/output/metrics_comparison.png
